# Flow latents

In [1]:
# imports
from importlib import reload
import os
os.environ['ENDPOINT_URL'] = 'https://s3.nautilus.optiputer.net'

import numpy as np

import h5py 
import torch
from tqdm.auto import tqdm

from ulmo import io as ulmo_io
from ulmo.models import io as models_io
from ulmo.utils import HDF5Dataset

Cartopy not installed.  Some plots will not work!


# Load model

In [2]:
pae = models_io.load_modis_l2()

Using device=cuda
Loading autoencoder model from: s3://modis-l2/Models/R2019_2010_128x128_std/autoencoder.pt
Loading flow model from: s3://modis-l2/Models/R2019_2010_128x128_std/flow.pt
scaler loaded from: s3://modis-l2/Models/R2019_2010_128x128_std/MODIS_R2019_2010_95clear_128x128_preproc_std_scaler.pkl


/opt/conda/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


# Load latents

In [3]:
latent_file = '/home/jovyan/Oceanography/SST-OOD/Test/Evaluations/MODIS_R2019_2012_95clear_128x128_latents_std.h5'
#latent_h5.keys()
latent_h5 = h5py.File(latent_file, 'r')
latents = latent_h5['latents'][:]

In [4]:
latents.shape

(730634, 512)

## Load LL

In [5]:
LL_file = '/home/jovyan/Oceanography/SST-OOD/Test/Evaluations/MODIS_R2019_2012_95clear_128x128_log_prob_std.h5'
LL_h5 = h5py.File(LL_file, 'r')
#LL_h5.keys()
LL = LL_h5['valid'][:]

In [6]:
LL[0]

44.542786

# Apply ulmo

## Data Loader

In [7]:
dset_l = torch.utils.data.TensorDataset(torch.from_numpy(latents[0:1]).float())
loader = torch.utils.data.DataLoader(
    dset_l, batch_size=1024, shuffle=False, 
    drop_last=False, num_workers=16)

## Log prob

In [8]:
#with h5py.File(output_file, 'w') as f:
with torch.no_grad():
    log_prob = [pae.flow.log_prob(data[0].to(pae.device)).detach().cpu().numpy()
            for data in tqdm(loader, total=len(loader), unit='batch', desc='Computing log probs')]

Computing log probs:   0%|          | 0/1 [00:02<?, ?batch/s]

In [9]:
log_prob[0]

array([43.5448], dtype=float32)

In [10]:
data = [data for data in loader]

In [11]:
data[0][0].shape

torch.Size([1, 512])

In [12]:
embedded_contet = pae.flow.flow._embedding_net(None)

In [13]:
noise, logabsdet = pae.flow.flow._transform(data[0][0].to(pae.device))

In [14]:
noise.shape

torch.Size([1, 512])

In [15]:
logabsdet

tensor([744.8929], device='cuda:0', grad_fn=<AddBackward0>)

In [16]:
embedded_contet

In [17]:
pae.flow.flow._distribution.log_prob(noise)

tensor([-699.0179], device='cuda:0', grad_fn=<SubBackward0>)

In [18]:
noise

tensor([[ 4.3123e-01, -9.1272e-01,  1.0792e+00,  8.3181e-01,  5.0570e-01,
         -8.2892e-02, -1.3332e+00, -7.5554e-01, -3.0799e-01,  5.8437e-01,
          5.9252e-01, -6.2341e-01, -1.1553e+00,  9.5788e-01, -1.0780e+00,
          1.1618e+00,  2.4692e-01,  2.9450e-02,  1.4103e-01,  1.0777e-01,
          4.8485e-01,  1.1857e+00, -9.2329e-01,  7.4356e-01, -2.7442e-01,
          1.5289e-01, -1.3573e+00,  7.0265e-02, -2.0161e-01,  5.7026e-01,
         -1.1215e+00,  1.6690e+00,  3.2713e-01,  1.0896e+00, -3.5509e-02,
          1.9196e-01,  7.8239e-01,  7.4307e-01,  3.3368e-01, -1.5176e+00,
         -2.7391e-01,  1.9960e-02, -1.2298e+00,  5.4875e-01, -7.8387e-01,
         -2.4160e-01,  5.4403e-01, -1.6633e+00, -1.1140e+00,  6.9872e-01,
          1.0181e+00, -2.7798e-01, -4.8325e-01,  3.0729e-02,  3.4740e-01,
          6.5682e-01,  7.1380e-01, -8.1244e-01,  1.4045e+00,  6.9460e-01,
         -1.5997e-01,  1.0378e+00,  1.4292e-01, -5.3882e-01, -2.0117e+00,
         -7.0843e-01, -5.2587e-01,  5.

----

# Testing

In [7]:
pp_file = '/home/jovyan/Oceanography/SST-OOD/Test/PreProc/MODIS_R2019_2012_95clear_128x128_preproc_std.h5'

In [11]:
pae.eval_data_file(pp_file, 'valid', '/home/jovyan/Oceanography/SST-OOD/Test/Evaluations/tmp_log_prob.h5', csv=False)

Calculating latents..
Scaling..
Probabilities now


Computing log probs:   0%|          | 0/714 [00:02<?, ?batch/s]

Wrote latents to /home/jovyan/Oceanography/SST-OOD/Test/Evaluations/tmp_latents.h5
Log probabilities saved to /home/jovyan/Oceanography/SST-OOD/Test/Evaluations/tmp_log_prob.h5.


array([ 44.542786, 340.43127 , 390.7146  , ..., 108.63623 , 295.42798 ,
       334.82202 ], dtype=float32)

In [12]:
latent_file2 = '/home/jovyan/Oceanography/SST-OOD/Test/Evaluations/tmp_latents.h5'
#latent_h5.keys()
latent_h5_2 = h5py.File(latent_file2, 'r')
latents2 = latent_h5_2['latents'][:]

In [18]:
np.all(np.isclose(latents2[0], latents[0]))

True